In [1]:

import os
import csv
import json
import numpy as np
from PIL import Image
from scipy.ndimage import binary_erosion
import os
from pathlib import Path
notebook_dir = Path.cwd()
print('original',notebook_dir)
proj_root = notebook_dir.parent
BASE_DIR = str(proj_root / "CRYO-SEM DATA" / "CRYO-SEM X30000" / "CRYO-SEM X30000 [1]")

# This script:
# 1. Reads GOLD STANDARD.tif and each method .tif from BASE_DIR
# 2. Converts them to binary pore masks (1=pore, 0=background)
# 3. Calculates Dice, IoU, MCC, etc.
# 4. Measures pore fraction and pore area in µm²
# 5. Saves:
#       - per-method overlay TIFFs
#       - per-method CSVs
#       - one summary CSV for all methods
#       - a copy of this code (.py)
#       - a minimal notebook (.ipynb)
#
# All outputs go into your GitHub repo folder.

# BASE_DIR = r"C:\Users\walsh\Documents\GitHub\AGAROSE-HYDROGEL-TRENDS-USING-AI-ML\CRYO-SEM DATA\CRYO-SEM X30000\CRYO-SEM X30000 [1]"
GT_FILENAME = "GOLD STANDARD.tif"

GITHUB_DIR = proj_root
GITHUB_DIR = str(GITHUB_DIR)
os.makedirs(GITHUB_DIR, exist_ok=True)

METRICS_CSV = os.path.join(GITHUB_DIR, "metrics_summary.csv")

# pixel calibration: 640 x 480 image covers 5.98 µm x 4.49 µm
PIX_SIZE_X_UM = 5.98 / 640.0
PIX_SIZE_Y_UM = 4.49 / 480.0
PIX_AREA_UM2  = PIX_SIZE_X_UM * PIX_SIZE_Y_UM


def _read_tiff_any(path):
    """Read a TIFF from disk using Pillow. Return as numpy array."""
    if not os.path.exists(path):
        return None
    with Image.open(path) as im:
        im.load()
        # convert to grayscale 16-bit or 8-bit
        if "I;16" in im.mode:
            im = im.convert("I;16")
        else:
            im = im.convert("L")
        arr = np.array(im)
    return arr


def _to_gray(arr):
    """Ensure we have a single 2D grayscale array."""
    if arr is None:
        return None
    if arr.ndim == 2:
        return arr
    if arr.ndim == 3:
        # if RGB slipped in, average channels
        if arr.shape[2] >= 3:
            return np.mean(arr[:, :, :3], axis=2).astype(arr.dtype)
        else:
            return arr[:, :, 0]
    return arr


def _ensure_uint(arr):
    """Force array into uint8 or uint16, for consistent thresholding."""
    if arr.dtype == np.uint8 or arr.dtype == np.uint16:
        return arr

    if np.issubdtype(arr.dtype, np.floating):
        a_min = float(arr.min())
        a_max = float(arr.max())
        rng = (a_max - a_min) + 1e-12
        scaled = (arr - a_min) / rng
        scaled = (scaled * 255.0 + 0.5).astype(np.uint8)
        return scaled

    maxv = float(arr.max())
    if maxv > 255.0:
        return arr.astype(np.uint16)
    return arr.astype(np.uint8)


def otsu_thresh_uint8(gray_u8):
    """Manual Otsu threshold. Returns mask of dark pixels as 1."""
    hist = np.bincount(gray_u8.flatten(), minlength=256).astype(float)
    total = gray_u8.size
    prob = hist / float(total)

    cum_prob = np.cumsum(prob)
    cum_mean = np.cumsum(prob * np.arange(256))
    global_mean = cum_mean[-1]

    best_t = 0
    best_score = -1.0

    for t in range(256):
        w0 = cum_prob[t]
        w1 = 1.0 - w0
        if w0 == 0.0 or w1 == 0.0:
            continue
        mu0 = cum_mean[t] / w0
        mu1 = (global_mean - cum_mean[t]) / w1
        diff = mu0 - mu1
        score = w0 * w1 * diff * diff
        if score > best_score:
            best_score = score
            best_t = t

    # pores are darker, so <= threshold is pore
    mask_dark = (gray_u8 <= best_t).astype(np.uint8)
    return mask_dark


def binarize_pores_black(img_gray):
    """Make a binary mask where pores (dark) are 1 and background is 0."""
    g = _ensure_uint(img_gray)
    uvals = np.unique(g)

    # common case where the mask is already binary grayscale
    if uvals.size == 2:
        darker = int(uvals[0])
        pores = (g == darker)
        return pores.astype(np.uint8)

    # not binary already: do Otsu on 8-bit version
    if g.dtype == np.uint16:
        g8 = (g / 257).astype(np.uint8)
    else:
        g8 = g.astype(np.uint8)

    pores = otsu_thresh_uint8(g8)
    return pores


def read_mask_as_binary(path):
    """Load tiff, turn into a binary pore mask (0/1)."""
    raw = _read_tiff_any(path)
    if raw is None:
        raise FileNotFoundError("Cannot read TIFF: " + path)
    gray = _to_gray(raw)
    if gray is None or gray.ndim != 2:
        raise ValueError("Not single-channel grayscale: " + path)
    return binarize_pores_black(gray)


def _safe_div(n, d):
    if d == 0:
        return 0.0
    return float(n) / float(d)


def calc_confusion(gt, pr):
    """Return tp, fp, tn, fn for two binary masks."""
    gt = gt.astype(np.uint8)
    pr = pr.astype(np.uint8)

    tp = int(np.sum((gt == 1) & (pr == 1)))
    tn = int(np.sum((gt == 0) & (pr == 0)))
    fp = int(np.sum((gt == 0) & (pr == 1)))
    fn = int(np.sum((gt == 1) & (pr == 0)))

    return tp, fp, tn, fn


def compute_metrics(gt, pr):
    """Return a dict of Dice, IoU, MCC, etc."""
    tp, fp, tn, fn = calc_confusion(gt, pr)

    acc  = _safe_div(tp + tn, tp + tn + fp + fn)
    prec = _safe_div(tp, tp + fp)
    rec  = _safe_div(tp, tp + fn)
    spec = _safe_div(tn, tn + fp)
    ba   = 0.5 * (rec + spec)
    dice = _safe_div(2 * tp, 2 * tp + fp + fn)
    iou  = _safe_div(tp, tp + fp + fn)

    # MCC denominator
    prod_val = float((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    if prod_val > 0.0:
        den = prod_val ** 0.5
    else:
        den = 0.0
    if den > 0.0:
        mcc_val = (tp * tn - fp * fn) / den
    else:
        mcc_val = 0.0

    out = {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "specificity": spec,
        "balanced_accuracy": ba,
        "f1_dice": dice,
        "iou_jaccard": iou,
        "mcc": mcc_val,
        "TP": tp,
        "FP": fp,
        "TN": tn,
        "FN": fn
    }
    return out


def pore_fraction(mask):
    """Fraction of pixels that are pore=1."""
    return float(np.mean(mask))


def pore_area_um2(mask):
    """Total pore area in square microns."""
    pore_px = int(np.sum(mask == 1))
    return pore_px * PIX_AREA_UM2


def make_pil_overlay(gt_mask, pr_mask, save_path):
    """
    Make an overlay using Pillow .paste(mask=...),
    similar to the GeeksforGeeks example.

    Colors:
      red   = false positive (pred says pore, GT says no pore)
      blue  = false negative (pred missed pore GT has)
      green = outline of true positive
    """
    base_gray = (1 - gt_mask) * 255.0
    base_gray = base_gray.astype(np.uint8)
    base_img = Image.fromarray(base_gray, mode="L").convert("RGBA")

    h, w = gt_mask.shape

    # false positives
    fp_mask = ((gt_mask == 0) & (pr_mask == 1)).astype(np.uint8) * 255
    red_img = Image.new("RGBA", (w, h), (255, 0, 0, 180))
    red_mask = Image.fromarray(fp_mask.astype(np.uint8), mode="L")
    base_img.paste(red_img, (0, 0), mask=red_mask)

    # false negatives
    fn_mask = ((gt_mask == 1) & (pr_mask == 0)).astype(np.uint8) * 255
    blue_img = Image.new("RGBA", (w, h), (0, 0, 255, 180))
    blue_mask = Image.fromarray(fn_mask.astype(np.uint8), mode="L")
    base_img.paste(blue_img, (0, 0), mask=blue_mask)

    # true positives, outline only
    tp_region = ((gt_mask == 1) & (pr_mask == 1)).astype(np.uint8)
    tp_eroded = binary_erosion(tp_region, border_value=0)
    tp_edge = tp_region.astype(np.uint8) - tp_eroded.astype(np.uint8)
    tp_edge_mask = (tp_edge > 0).astype(np.uint8) * 255

    green_img = Image.new("RGBA", (w, h), (0, 255, 0, 255))
    green_mask = Image.fromarray(tp_edge_mask.astype(np.uint8), mode="L")
    base_img.paste(green_img, (0, 0), mask=green_mask)

    # save RGB TIFF
    final_rgb = base_img.convert("RGB")
    final_rgb.save(save_path, format="TIFF")
    return save_path


def write_notebook_copy(code_text, ipynb_out_path):
    """Save a tiny 1-cell .ipynb that just contains this script."""
    nb_obj = {
        "cells": [
            {
                "cell_type": "code",
                "execution_count": None,
                "metadata": {},
                "outputs": [],
                "source": code_text.splitlines(True)
            }
        ],
        "metadata": {
            "language_info": {"name": "python"},
            "kernelspec": {
                "display_name": "Python",
                "language": "python",
                "name": "python"
            }
        },
        "nbformat": 4,
        "nbformat_minor": 5
    }

    with open(ipynb_out_path, "w", encoding="utf-8") as f_out:
        json.dump(nb_obj, f_out, indent=2)


def write_single_method_csv(row_dict, repo_dir):
    """
    Save metrics for a single method as <method>_metrics.csv
    so each method is traceable in Git.
    """
    method_name = row_dict["method"]
    out_path = os.path.join(repo_dir, method_name + "_metrics.csv")

    cols = [
        "method",
        "f1_dice",
        "iou_jaccard",
        "mcc",
        "precision",
        "recall",
        "specificity",
        "balanced_accuracy",
        "accuracy",
        "TP",
        "FP",
        "TN",
        "FN",
        "gt_pore_fraction",
        "pred_pore_fraction",
        "pore_fraction_bias",
        "gt_pore_area_um2",
        "pred_pore_area_um2",
        "pore_area_bias_um2"
    ]

    with open(out_path, "w", newline="", encoding="utf-8") as f_one:
        w = csv.DictWriter(f_one, fieldnames=cols)
        w.writeheader()
        w.writerow(row_dict)

    return out_path


def analyze_folder(folder, repo_dir, code_text):
    """
    Do the full run:
    - read GT
    - loop over each method .tif
    - compute metrics and pore stats
    - save overlay.tif
    - save <method>_metrics.csv
    - save metrics_summary.csv
    - save cryo_sem_analysis.py and cryo_sem_analysis.ipynb
    """

    gt_path = os.path.join(folder, GT_FILENAME)
    if not os.path.exists(gt_path):
        raise FileNotFoundError("Ground truth not found: " + gt_path)

    gt_mask = read_mask_as_binary(gt_path)

    all_rows = []

    for fname in os.listdir(folder):
        low = fname.lower()
        if not (low.endswith(".tif") or low.endswith(".tiff")):
            continue
        if fname == GT_FILENAME:
            continue

        pr_path = os.path.join(folder, fname)
        pr_mask = read_mask_as_binary(pr_path)

        if gt_mask.shape != pr_mask.shape:
            raise ValueError("Shape mismatch: GT " + str(gt_mask.shape) +
                             " vs " + fname + " " + str(pr_mask.shape))

        m = compute_metrics(gt_mask, pr_mask)

        gt_frac = pore_fraction(gt_mask)
        pr_frac = pore_fraction(pr_mask)
        frac_bias = pr_frac - gt_frac

        gt_area = pore_area_um2(gt_mask)
        pr_area = pore_area_um2(pr_mask)
        area_bias = pr_area - gt_area

        method_name = os.path.splitext(fname)[0]

        overlay_file = os.path.join(repo_dir, method_name + "_overlay.tif")
        make_pil_overlay(gt_mask, pr_mask, overlay_file)

        row = {
            "method": method_name,
            "f1_dice": m["f1_dice"],
            "iou_jaccard": m["iou_jaccard"],
            "mcc": m["mcc"],
            "precision": m["precision"],
            "recall": m["recall"],
            "specificity": m["specificity"],
            "balanced_accuracy": m["balanced_accuracy"],
            "accuracy": m["accuracy"],
            "TP": m["TP"],
            "FP": m["FP"],
            "TN": m["TN"],
            "FN": m["FN"],
            "gt_pore_fraction": gt_frac,
            "pred_pore_fraction": pr_frac,
            "pore_fraction_bias": frac_bias,
            "gt_pore_area_um2": gt_area,
            "pred_pore_area_um2": pr_area,
            "pore_area_bias_um2": area_bias
        }

        all_rows.append(row)

        single_csv_path = write_single_method_csv(row, repo_dir)

        print(method_name + ": Dice=%.3f IoU=%.3f Bias=%.2f%% -> %s / %s"
              % (m["f1_dice"], m["iou_jaccard"],
                 frac_bias * 100.0, overlay_file, single_csv_path))

    # now write combined CSV for all methods
    if len(all_rows) > 0:
        fieldnames = [
            "method",
            "f1_dice",
            "iou_jaccard",
            "mcc",
            "precision",
            "recall",
            "specificity",
            "balanced_accuracy",
            "accuracy",
            "TP",
            "FP",
            "TN",
            "FN",
            "gt_pore_fraction",
            "pred_pore_fraction",
            "pore_fraction_bias",
            "gt_pore_area_um2",
            "pred_pore_area_um2",
            "pore_area_bias_um2"
        ]
    else:
        fieldnames = []

    with open(METRICS_CSV, "w", newline="", encoding="utf-8") as f_all:
        w_all = csv.DictWriter(f_all, fieldnames=fieldnames)
        w_all.writeheader()
        for r in all_rows:
            w_all.writerow(r)

    print("metrics_summary.csv -> " + METRICS_CSV)

    # save a copy of the code and a notebook version into the repo
    script_path_out = os.path.join(repo_dir, "cryo_sem_analysis.py")
    nb_path_out     = os.path.join(repo_dir, "cryo_sem_analysis.ipynb")

    with open(script_path_out, "w", encoding="utf-8") as f_py:
        f_py.write(code_text)

    write_notebook_copy(code_text, nb_path_out)

    print("saved: " + script_path_out)
    print("saved: " + nb_path_out)
    print("overlays and per-method metrics saved in: " + repo_dir)


# run it immediately when this script is executed
analyze_folder(BASE_DIR, GITHUB_DIR, "")


original c:\Users\walsh\Documents\GitHub\AGAROSE-HYDROGEL-TRENDS-USING-AI-ML\CODE
60%: Dice=0.717 IoU=0.559 Bias=20.33% -> c:\Users\walsh\Documents\GitHub\AGAROSE-HYDROGEL-TRENDS-USING-AI-ML\60%_overlay.tif / c:\Users\walsh\Documents\GitHub\AGAROSE-HYDROGEL-TRENDS-USING-AI-ML\60%_metrics.csv
FREEHAND: Dice=0.610 IoU=0.439 Bias=-11.81% -> c:\Users\walsh\Documents\GitHub\AGAROSE-HYDROGEL-TRENDS-USING-AI-ML\FREEHAND_overlay.tif / c:\Users\walsh\Documents\GitHub\AGAROSE-HYDROGEL-TRENDS-USING-AI-ML\FREEHAND_metrics.csv
ILASTIK: Dice=0.723 IoU=0.567 Bias=-5.10% -> c:\Users\walsh\Documents\GitHub\AGAROSE-HYDROGEL-TRENDS-USING-AI-ML\ILASTIK_overlay.tif / c:\Users\walsh\Documents\GitHub\AGAROSE-HYDROGEL-TRENDS-USING-AI-ML\ILASTIK_metrics.csv
OTSU: Dice=0.624 IoU=0.453 Bias=-8.47% -> c:\Users\walsh\Documents\GitHub\AGAROSE-HYDROGEL-TRENDS-USING-AI-ML\OTSU_overlay.tif / c:\Users\walsh\Documents\GitHub\AGAROSE-HYDROGEL-TRENDS-USING-AI-ML\OTSU_metrics.csv
OVAL: Dice=0.653 IoU=0.484 Bias=-2.66% -> c

In [2]:
import os
import csv
import json
import numpy as np
from PIL import Image
from scipy.ndimage import binary_erosion
import math
import os
from pathlib import Path
notebook_dir = Path.cwd()
print('original',notebook_dir)
proj_root = notebook_dir.parent
BASE_DIR = str(proj_root / "CRYO-SEM DATA" / "CRYO-SEM X30000" / "CRYO-SEM X30000 [1]")

# script to compare different pore detection methods against gold standard
# loads images, makes binary masks, calculates accuracy metrics
# saves overlay images and CSV files with results

#data_folder = r"C:\Users\walsh\Documents\GitHub\AGAROSE-HYDROGEL-TRENDS-USING-AI-ML\CRYO-SEM DATA\CRYO-SEM X30000\CRYO-SEM X30000 [1]"
data_folder = BASE_DIR
gold_standard_file = "GOLD STANDARD.tif"

#output_folder = r"C:\Users\walsh\Documents\GitHub\AGAROSE-HYDROGEL-TRENDS-USING-AI-ML\OVERLAYS"
output_folder = os.path.join(proj_root,'OVERLAYS')
os.makedirs(output_folder, exist_ok=True)

summary_csv_file = os.path.join(output_folder, "metrics_summary.csv")

# image size is 640x480 pixels covering 5.98 x 4.49 micrometers
pixel_width_um = 5.98 / 640.0
pixel_height_um = 4.49 / 480.0
pixel_area_um2 = pixel_width_um * pixel_height_um

def load_tiff_image(file_path):
    """load a tiff file and return as numpy array"""
    if not os.path.exists(file_path):
        return None
    
    image = Image.open(file_path)
    # convert to grayscale
    if "I;16" in image.mode:
        image = image.convert("I;16")
    else:
        image = image.convert("L")
    
    array_data = np.array(image)
    image.close()
    return array_data

def make_grayscale(image_array):
    """make sure image is 2D grayscale"""
    if image_array is None:
        return None
    if len(image_array.shape) == 2:
        return image_array
    if len(image_array.shape) == 3:
        # if color image, average the channels
        if image_array.shape[2] >= 3:
            gray_image = np.mean(image_array[:, :, :3], axis=2)
            return gray_image.astype(image_array.dtype)
        else:
            return image_array[:, :, 0]
    return image_array

def convert_to_uint8(image_array):
    """convert image to 8-bit format for processing"""
    if image_array.dtype == np.uint8:
        return image_array
    
    # if floating point values, scale to 0-255
    if np.issubdtype(image_array.dtype, np.floating):
        min_val = float(image_array.min())
        max_val = float(image_array.max())
        range_val = max_val - min_val
        if range_val > 0:
            scaled = (image_array - min_val) / range_val
            scaled = (scaled * 255.0).astype(np.uint8)
            return scaled
        else:
            return np.zeros_like(image_array, dtype=np.uint8)
    
    # if 16-bit, scale down to 8-bit
    if image_array.dtype == np.uint16:
        scaled = (image_array / 257).astype(np.uint8)
        return scaled
    
    return image_array.astype(np.uint8)

def find_threshold_otsu(gray_image):
    """find best threshold using otsu method"""
    # count how many pixels at each brightness level
    histogram = np.bincount(gray_image.flatten(), minlength=256)
    total_pixels = gray_image.size
    
    # convert to probabilities
    probabilities = histogram / float(total_pixels)
    
    best_threshold = 0
    best_variance = 0.0
    
    # try each possible threshold
    for threshold in range(256):
        # calculate weights for background and foreground
        weight_background = np.sum(probabilities[:threshold+1])
        weight_foreground = 1.0 - weight_background
        
        if weight_background == 0 or weight_foreground == 0:
            continue
        
        # calculate mean brightness for each group
        mean_background = np.sum(probabilities[:threshold+1] * np.arange(threshold+1)) / weight_background
        mean_foreground = np.sum(probabilities[threshold+1:] * np.arange(threshold+1, 256)) / weight_foreground
        
        # calculate between-class variance
        variance = weight_background * weight_foreground * (mean_background - mean_foreground) ** 2
        
        if variance > best_variance:
            best_variance = variance
            best_threshold = threshold
    
    return best_threshold

def make_binary_mask(gray_image):
    """convert grayscale image to binary pore mask"""
    # convert to 8-bit if needed
    processed_image = convert_to_uint8(gray_image)
    
    # check if already binary
    unique_values = np.unique(processed_image)
    if len(unique_values) == 2:
        # already binary, assume darker pixels are pores
        darker_value = unique_values[0]
        pore_mask = (processed_image == darker_value).astype(np.uint8)
        return pore_mask
    
    # find threshold automatically
    threshold = find_threshold_otsu(processed_image)
    
    # pores are darker pixels
    pore_mask = (processed_image <= threshold).astype(np.uint8)
    return pore_mask

def load_image_as_binary(file_path):
    """load tiff file and convert to binary pore mask"""
    raw_image = load_tiff_image(file_path)
    if raw_image is None:
        raise FileNotFoundError("Could not read image: " + file_path)
    
    gray_image = make_grayscale(raw_image)
    if gray_image is None:
        raise ValueError("Could not convert to grayscale: " + file_path)
    
    binary_mask = make_binary_mask(gray_image)
    return binary_mask

def safe_divide(numerator, denominator):
    """divide two numbers safely, return 0 if denominator is 0"""
    if denominator == 0:
        return 0.0
    return float(numerator) / float(denominator)

def calculate_confusion_matrix(true_mask, predicted_mask):
    """calculate true positive, false positive, etc"""
    true_binary = true_mask.astype(np.uint8)
    pred_binary = predicted_mask.astype(np.uint8)
    
    # count pixels in each category
    true_positive = int(np.sum((true_binary == 1) & (pred_binary == 1)))
    true_negative = int(np.sum((true_binary == 0) & (pred_binary == 0)))
    false_positive = int(np.sum((true_binary == 0) & (pred_binary == 1)))
    false_negative = int(np.sum((true_binary == 1) & (pred_binary == 0)))
    
    return true_positive, false_positive, true_negative, false_negative

def calculate_accuracy_metrics(true_mask, predicted_mask):
    """calculate dice, IoU, precision, recall etc"""
    tp, fp, tn, fn = calculate_confusion_matrix(true_mask, predicted_mask)
    
    # basic metrics
    accuracy = safe_divide(tp + tn, tp + tn + fp + fn)
    precision = safe_divide(tp, tp + fp)
    recall = safe_divide(tp, tp + fn)
    specificity = safe_divide(tn, tn + fp)
    
    # combined metrics
    dice_score = safe_divide(2 * tp, 2 * tp + fp + fn)
    iou_score = safe_divide(tp, tp + fp + fn)
    
    # calculate MCC (correlation coefficient) - FIXED
    mcc_denominator = (tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)
    if mcc_denominator > 0:
        mcc_value = (tp * tn - fp * fn) / math.sqrt(float(mcc_denominator))
    else:
        mcc_value = 0.0
    
    results = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "specificity": specificity,
        "dice": dice_score,
        "iou": iou_score,
        "mcc": mcc_value,
        "true_positive": tp,
        "false_positive": fp,
        "true_negative": tn,
        "false_negative": fn
    }
    
    return results

def calculate_pore_fraction(mask):
    """what fraction of pixels are pores"""
    return float(np.mean(mask))

def calculate_pore_area_micrometers(mask):
    """total pore area in square micrometers"""
    pore_pixels = int(np.sum(mask == 1))
    area_um2 = pore_pixels * pixel_area_um2
    return area_um2

def create_overlay_image(true_mask, predicted_mask, output_path):
    """create colored overlay showing differences between masks"""
    # start with gray background
    background = (1 - true_mask) * 255.0
    background = background.astype(np.uint8)
    base_image = Image.fromarray(background, mode="L").convert("RGBA")
    
    height, width = true_mask.shape
    
    # false positives in red (predicted pore but not true pore)
    false_pos = ((true_mask == 0) & (predicted_mask == 1)).astype(np.uint8) * 255
    red_layer = Image.new("RGBA", (width, height), (255, 0, 0, 180))
    red_mask = Image.fromarray(false_pos, mode="L")
    base_image.paste(red_layer, (0, 0), mask=red_mask)
    
    # false negatives in blue (missed true pores)
    false_neg = ((true_mask == 1) & (predicted_mask == 0)).astype(np.uint8) * 255
    blue_layer = Image.new("RGBA", (width, height), (0, 0, 255, 180))
    blue_mask = Image.fromarray(false_neg, mode="L")
    base_image.paste(blue_layer, (0, 0), mask=blue_mask)
    
    # true positives in green outline only
    true_pos_region = ((true_mask == 1) & (predicted_mask == 1)).astype(np.uint8)
    true_pos_eroded = binary_erosion(true_pos_region)
    true_pos_outline = true_pos_region - true_pos_eroded.astype(np.uint8)
    outline_mask = (true_pos_outline > 0).astype(np.uint8) * 255
    
    green_layer = Image.new("RGBA", (width, height), (0, 255, 0, 255))
    green_mask = Image.fromarray(outline_mask, mode="L")
    base_image.paste(green_layer, (0, 0), mask=green_mask)
    
    # save as RGB image
    final_image = base_image.convert("RGB")
    final_image.save(output_path)
    return output_path

def save_method_csv(results_dict, output_dir):
    """save results for one method to its own CSV file"""
    method_name = results_dict["method"]
    csv_filename = method_name + "_metrics.csv"
    csv_path = os.path.join(output_dir, csv_filename)
    
    column_names = [
        "method", "dice", "iou", "mcc", "precision", "recall", 
        "specificity", "accuracy", "true_positive", "false_positive", 
        "true_negative", "false_negative", "true_pore_fraction", 
        "pred_pore_fraction", "pore_fraction_diff", "true_pore_area_um2", 
        "pred_pore_area_um2", "pore_area_diff_um2"
    ]
    
    with open(csv_path, "w", newline="") as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=column_names)
        writer.writeheader()
        writer.writerow(results_dict)
    
    return csv_path

def create_notebook_file(python_code, notebook_path):
    """save python code as jupyter notebook"""
    notebook_data = {
        "cells": [{
            "cell_type": "code",
            "execution_count": None,
            "metadata": {},
            "outputs": [],
            "source": python_code.splitlines(True)
        }],
        "metadata": {
            "language_info": {"name": "python"},
            "kernelspec": {"display_name": "Python", "language": "python", "name": "python"}
        },
        "nbformat": 4,
        "nbformat_minor": 5
    }
    
    with open(notebook_path, "w") as notebook_file:
        json.dump(notebook_data, notebook_file, indent=2)

def process_all_images(input_folder, output_folder):
    """main function to process all images and create results"""
    
    # load gold standard image
    gold_standard_path = os.path.join(input_folder, gold_standard_file)
    if not os.path.exists(gold_standard_path):
        raise FileNotFoundError("Gold standard file not found: " + gold_standard_path)
    
    true_mask = load_image_as_binary(gold_standard_path)
    all_results = []
    
    # process each method image
    for filename in os.listdir(input_folder):
        filename_lower = filename.lower()
        if not (filename_lower.endswith(".tif") or filename_lower.endswith(".tiff")):
            continue
        if filename == gold_standard_file:
            continue
        
        # load and process this method's image
        method_path = os.path.join(input_folder, filename)
        predicted_mask = load_image_as_binary(method_path)
        
        # check image sizes match
        if true_mask.shape != predicted_mask.shape:
            error_msg = "Image sizes don't match: " + str(true_mask.shape) + " vs " + str(predicted_mask.shape)
            raise ValueError(error_msg)
        
        # calculate accuracy metrics
        metrics = calculate_accuracy_metrics(true_mask, predicted_mask)
        
        # calculate pore statistics
        true_fraction = calculate_pore_fraction(true_mask)
        pred_fraction = calculate_pore_fraction(predicted_mask)
        fraction_difference = pred_fraction - true_fraction
        
        true_area = calculate_pore_area_micrometers(true_mask)
        pred_area = calculate_pore_area_micrometers(predicted_mask)
        area_difference = pred_area - true_area
        
        # get method name from filename
        method_name = os.path.splitext(filename)[0]
        
        # create overlay image
        overlay_filename = method_name + "_overlay.tif"
        overlay_path = os.path.join(output_folder, overlay_filename)
        create_overlay_image(true_mask, predicted_mask, overlay_path)
        
        # compile all results for this method
        method_results = {
            "method": method_name,
            "dice": metrics["dice"],
            "iou": metrics["iou"],
            "mcc": metrics["mcc"],
            "precision": metrics["precision"],
            "recall": metrics["recall"],
            "specificity": metrics["specificity"],
            "accuracy": metrics["accuracy"],
            "true_positive": metrics["true_positive"],
            "false_positive": metrics["false_positive"],
            "true_negative": metrics["true_negative"],
            "false_negative": metrics["false_negative"],
            "true_pore_fraction": true_fraction,
            "pred_pore_fraction": pred_fraction,
            "pore_fraction_diff": fraction_difference,
            "true_pore_area_um2": true_area,
            "pred_pore_area_um2": pred_area,
            "pore_area_diff_um2": area_difference
        }
        
        all_results.append(method_results)
        
        # save individual method CSV
        method_csv_path = save_method_csv(method_results, output_folder)
        
        # print progress
        print(method_name + ": Dice=" + str(round(metrics["dice"], 3)) + 
              " IoU=" + str(round(metrics["iou"], 3)) + 
              " Bias=" + str(round(fraction_difference * 100, 2)) + "%")
    
    # save combined results CSV
    if len(all_results) > 0:
        column_names = [
            "method", "dice", "iou", "mcc", "precision", "recall", 
            "specificity", "accuracy", "true_positive", "false_positive", 
            "true_negative", "false_negative", "true_pore_fraction", 
            "pred_pore_fraction", "pore_fraction_diff", "true_pore_area_um2", 
            "pred_pore_area_um2", "pore_area_diff_um2"
        ]
        
        with open(summary_csv_file, "w", newline="") as summary_file:
            writer = csv.DictWriter(summary_file, fieldnames=column_names)
            writer.writeheader()
            for result in all_results:
                writer.writerow(result)
    
    print("Summary CSV saved: " + summary_csv_file)
    
    # save copies of code
    script_output_path = os.path.join(output_folder, "pore_analysis.py")
    notebook_output_path = os.path.join(output_folder, "pore_analysis.ipynb")
    
    # get source code of this script
    try:
        script_source = ""
        # read this file to get the source code
        current_file_path = __file__
        with open(current_file_path, "r") as source_file:
            script_source = source_file.read()
    except:
        script_source = "# source code not available in this environment"
    
    with open(script_output_path, "w") as script_file:
        script_file.write(script_source)
    
    create_notebook_file(script_source, notebook_output_path)
    
    print("Code saved: " + script_output_path)
    print("Notebook saved: " + notebook_output_path)

# run the analysis
process_all_images(data_folder, output_folder)

original c:\Users\walsh\Documents\GitHub\AGAROSE-HYDROGEL-TRENDS-USING-AI-ML\CODE
60%: Dice=0.717 IoU=0.559 Bias=20.33%
FREEHAND: Dice=0.61 IoU=0.439 Bias=-11.81%
ILASTIK: Dice=0.723 IoU=0.567 Bias=-5.1%
OTSU: Dice=0.624 IoU=0.453 Bias=-8.47%
OVAL: Dice=0.653 IoU=0.484 Bias=-2.66%
PLANKSTER: Dice=0.579 IoU=0.407 Bias=-16.9%
PORED2: Dice=0.655 IoU=0.487 Bias=23.37%
SAMJ: Dice=0.207 IoU=0.115 Bias=-35.13%
SEMI: Dice=0.411 IoU=0.258 Bias=-28.41%
UNET: Dice=0.657 IoU=0.49 Bias=1.62%
Summary CSV saved: c:\Users\walsh\Documents\GitHub\AGAROSE-HYDROGEL-TRENDS-USING-AI-ML\OVERLAYS\metrics_summary.csv
Code saved: c:\Users\walsh\Documents\GitHub\AGAROSE-HYDROGEL-TRENDS-USING-AI-ML\OVERLAYS\pore_analysis.py
Notebook saved: c:\Users\walsh\Documents\GitHub\AGAROSE-HYDROGEL-TRENDS-USING-AI-ML\OVERLAYS\pore_analysis.ipynb
